In [12]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib import style
import datetime as dt1
from datetime import datetime as dt

In [7]:
from subprocess import check_output

In [27]:
import subprocess as subp
print(subp.check_output(["dir","C:\\Users\\ReDI\\JUPYTER-LAB\\Yoon-Choi\\Stock-Price-Prediction-RNN-LSTM-GRU\\input\\"],shell=True).decode("utf8"))

 Volume in drive C is OS
 Volume Serial Number is ECFE-798C

 Directory of C:\Users\ReDI\JUPYTER-LAB\Yoon-Choi\Stock-Price-Prediction-RNN-LSTM-GRU\input

09/28/2021  07:44 PM    <DIR>          .
09/28/2021  07:44 PM    <DIR>          ..
09/28/2021  10:14 AM         1,414,934 fundamentals.csv
09/27/2021  01:11 PM        52,701,226 prices-split-adjusted.csv
09/28/2021  10:14 AM        51,667,341 prices.csv
09/27/2021  01:09 PM            61,381 securities.csv
               4 File(s)    105,844,882 bytes
               2 Dir(s)  42,624,507,904 bytes free



In [28]:
df=pd.read_csv('C:\\Users\\ReDI\\JUPYTER-LAB\\Yoon-Choi\\Stock-Price-Prediction-RNN-LSTM-GRU\\input\\prices.csv')
df.tail()
df=df.loc[df['symbol'] == 'CSCO']
print(df.tail())


              date symbol       open      close        low       high  \
848874  2016-12-23   CSCO  30.540001  30.530001  30.400000  30.600000   
849374  2016-12-27   CSCO  30.639999  30.680000  30.600000  30.820000   
849874  2016-12-28   CSCO  30.700001  30.420000  30.350000  30.770000   
850374  2016-12-29   CSCO  30.370001  30.459999  30.330000  30.549999   
850874  2016-12-30   CSCO  30.559999  30.219999  30.129999  30.600000   

            volume  
848874   9167500.0  
849374  14175500.0  
849874  12022200.0  
850374  10995600.0  
850874  20190000.0  


In [39]:
from sklearn import preprocessing, svm 
import math
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [37]:
forecast_col = 'close'
df.fillna(value=-99999, inplace=True)
forecast_out = int(math.ceil(0.01 * len(df)))
print(forecast_out)
df['label'] = df[forecast_col].shift(-forecast_out)
print(df.head())

18
            date symbol       open      close        low       high  \
356   2010-01-04   CSCO  24.110001  24.690001  24.010000  24.840000   
824   2010-01-05   CSCO  24.600000  24.580000  24.379999  24.730000   
1292  2010-01-06   CSCO  24.540001  24.420000  24.340000  24.740000   
1760  2010-01-07   CSCO  24.299999  24.530001  24.170000  24.570000   
2228  2010-01-08   CSCO  24.379999  24.660000  24.250000  24.700001   

          volume      label  
356   59853700.0  22.469999  
824   45124500.0  22.730000  
1292  35715700.0  23.020000  
1760  31531200.0  23.070000  
2228  39115900.0  23.160000  


In [42]:
#X = np.array(df.drop(['label'], 1))

X = np.array(df.drop(['label','symbol','date'], axis=1))

#print(X)
X = preprocessing.scale(X)

#print(X)
X_lately = X[-forecast_out:]

#print(X_lately)
X = X[:-forecast_out]

df.dropna(inplace=True)
y = np.array(df['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


ValueError: Found input variables with inconsistent numbers of samples: [1726, 1744]

In [43]:
clf = LinearRegression(n_jobs=-1)
clf.fit(X_train, y_train)
confidence = clf.score(X_test, y_test)
print(confidence)

NameError: name 'X_train' is not defined

In [44]:
forecast_set = clf.predict(X_lately)
print(forecast_set)
df['Forecast'] = np.nan
last_date = df.iloc[-1].date
print(last_date)
last_date=dt.strptime(last_date, '%Y-%m-%d').timestamp()
last_unix = last_date
one_day = 86400
next_unix = last_unix + one_day

for i in forecast_set:
    next_date = dt.fromtimestamp(next_unix)
    next_unix += 86400
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)]+[i]
print(df.tail())
df['close'].plot()
df['Forecast'].plot()
#plt.legend(loc=4)
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.